# Визуальный блокнот по использованию BERT для первого знакомства

*Credits: first part of this notebook is strongly based on Jay Alammar's [great blog post](http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/). His blog is a great way to dive into the DL and NLP concepts.*

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-sentence-classification.png" />

В этом блокноте мы будем использовать предварительно обученную модель глубокого обучения для обработки текста. Затем мы используем выходные данные этой модели для классификации текста. Текст представляет собой список предложений из рецензий на фильмы. И мы классифицируем каждое предложение как выражающее "позитивное" или "негативное" отношение к его предмету.

### Модели: Классификация тональности предложений

Наша цель — создать модель, которая принимает предложение (такое же, как в нашем наборе данных) и возвращает либо 1 (указывая на позитивную тональность предложения), либо 0 (указывая на негативную тональность). Мы можем представить это следующим образом:

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" />

На самом деле модель состоит из двух компонентов:

* `DistilBERT` обрабатывает предложение и передает извлеченную из него информацию следующей модели. `DistilBERT` — это уменьшенная версия `BERT`, разработанная и опубликованная командой `HuggingFace`. Это более легкая и быстрая версия `BERT`, которая примерно соответствует его производительности.

* Логистическая регрессия из библиотеки `scikit learn` принимает результат обработки `DistilBERT` и классифицирует предложение как позитивное или негативное (1 или 0 соответственно).

Данные, передаваемые между моделями, представляют собой вектор размером 768. Можно рассматривать этот вектор как эмбеддинг (векторное представление) предложения, который используется для классификации.

<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" />

### Dataset

Набор данных, который мы будем использовать в этом примере, - это [SST2](https://nlp.stanford.edu/sentiment/index.html), содержащий предложения из рецензий на фильмы, каждое из которых помечено как позитивное (со значением 1) или негативное (со значением 0):




<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>

### Установка библиотеки transformers

Давайте начнем с установки библиотеки `huggingface transformers`, чтобы мы могли загрузить нашу модель глубокого обучения для NLP.

In [1]:
!pip install -Uqq transformers

## 1. Использование BERT для классификации текста.

### 1.1. Загрузка предварительно обученной модели BERT.

Здесь мы будем использовать предварительно обученную модель `DistilBERT` из библиотеки `transformers`. 

Самый простой способ использовать такую модель — применить pipeline (конвейер). Это можно сделать следующим образом:

In [ ]:
from transformers import pipeline

unmasker = pipeline('fill-mask', 'distilbert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Однако у этого подхода есть недостатки: он не такой гибкий и не позволяет дообучить модель под свои задачи.

Поэтому мы пойдем другим путем и будем работать с моделью более "вручную". Для этого мы:

1. Загрузим саму модель и специальный инструмент для обработки текста (токенизатор)

2. Будем использовать их вместе для обработки данных

Вот как это работает: Сначала мы преобразуем текст в специальный числовой формат, который понимает модель, а затем извлечем из него нужные нам характеристики (признаки).

In [ ]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer, logging

logging.set_verbosity_error() # Игнорировать предупреждения при загрузке модели
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

text = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit.'
tokenized_text = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output = model(**tokenized_text)

output.last_hidden_state.shape

Объект `tokenizer` — это экземпляр `DistilBertTokenizer`, связанный с моделью `DistilBERT`.Он преобразует обычную строку (text) в числовой формат, который понимает модель.

За исключением части с `logging`, всё выглядит очень похоже на код из предыдущих практических блокнотов. Первое, что мы делаем, как и всегда, — токенизируем наш текст.

> **Примечание:** как вы можете видеть, мы использовали аргумент `return_tensors` в коде выше. Этот параметр указывает токенизатору преобразовать результат в тензоры PyTorch для использования с нашей моделью. Если мы не укажем этот параметр, мы получим точно такие же результаты, но упакованные в объекты python `list`.

Давайте рассмотрим этот шаг подробнее. Что именно возвращает `tokenizer.__call__`? Давайте разберемся:

In [ ]:
text

In [ ]:
tokenized_text

In [ ]:
tokenized_text = tokenizer(text)

for key, values in tokenized_text.items():
    values_type = type(values).__name__
    item_type = type(values[0]).__name__
    values_sample = f"[{', '.join(str(value) for value in values[:5])}, ...]"
    print(f"{key}: {values_type}[{item_type}]")
    print(f"length {len(values)}: {values_sample}\n")

Содержимое может отличаться для разных моделей, однако для токенизатора модели `DistilBert` возвращается объект, похожий на `dict`, с двумя списками Python под ключами `"input_ids"` и `"attention_mask"`. Оба списка имеют одинаковую длину, и маска внимания, кажется, состоит только из единиц.

Например, для строки

`text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit."`

токенайзер выполнит:

1. Разбиение на токены (словоподобные кусочки):
    
    `['lorem', 'ipsum', 'dolor', 'sit', 'amet', ',', 'consectetur', 'adipiscing', 'elit', '.']`


2. Добавление специальных токенов (если нужно) — например, [CLS] и [SEP] для BERT-подобных моделей.

3. Преобразование токенов в ID — каждое слово заменяется индексом из словаря DistilBERT:

    `'lorem' → 16204, 'ipsum' → 8920, ...`


4. Создание attention mask — где 1 = реальный токен, 0 = паддинг (если есть).

Мы разберемся с маской позже, а сейчас сосредоточимся на `"input_ids"` — это идентификаторы токенов для токенизированной последовательности. Давайте декодируем их, чтобы убедиться и посмотреть, что на самом деле происходит:

In [ ]:
tokenized_text['input_ids']

In [ ]:
print(f"Tokens: {tokenizer.convert_ids_to_tokens(tokenized_text['input_ids'])}")
print(f"Decoded sequence: '{tokenizer.decode(tokenized_text['input_ids'])}'")

Мы видим, что токенизатор на самом деле выполняет довольно много работы за кулисами: 
- он преобразует последовательность в нижний регистр (помните, мы используем `*-uncased` модель, что подразумевает отсутствие понимания заглавных букв), 
- добавляет специальные токены (`[CLS]` и `[SEP]`) 
- применяет BPE (Byte Pair Encoding). 

Именно так мы получаем 23 токена для такого небольшого текста.

[CLS] — вектор первого токена [CLS], который специально используется как “обобщённое представление” всего входного текста;

Этот вектор — свёртка смысла всего предложения.

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### 1.2. Загрузка набора данных

Однако работа с предложениями, отредактированными вручную, не представляет особого интереса. Давайте использовать нашу модель для работы с набором данных по классификации тональности. Мы воспользуемся библиотекой `pandas` для чтения набора данных и загрузки его в DataFrame.

In [ ]:
import pandas as pd


dataset_url = (
    'https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv'
)

dataset = pd.read_csv(dataset_url, delimiter='\t', header=None)
dataset.columns = ['text', 'label']
dataset.head()

Из-за соображений производительности мы будем использовать только 2000 предложений из набора данных.

In [10]:
dataset = # YOUR CODE

Мы можем попросить `pandas` показать, сколько предложений помечены как "позитивные" (значение 1) и сколько как "негативные" (значение 0).

In [ ]:
dataset['label'].value_counts()

### 1.3. Подготовка набора данных

Прежде чем мы сможем передать наши предложения в BERT, нам необходимо выполнить некоторую обработку, чтобы привести их в требуемый формат. Сначала давайте разделим наш `dataset` на отдельные `texts` и `labels`.

In [ ]:
texts =  # YOUR CODE
labels = # YOUR CODE

Теперь нам нужно токенизировать наши тексты.

In [ ]:
# YOUR CODE
# Tokenize the texts in dataset.
# Hint: our tokenizer can also work with lists of strings.
# tokenized_texts = ...

for key, values in tokenized_texts.items():
    values_type = type(values).__name__
    item_type = type(values[0]).__name__
    print(f"{key}: {values_type}[{item_type}], length {len(values)}")

Мы получили список списков. Однако нам нужен `torch.tensor`, чтобы использовать его с нашей моделью. Самое время вспомнить опцию `return_tensors`! 

Но если мы просто укажем ее, мы получим ошибку. Проблема здесь в том, что последовательности имеют разную длину:

In [ ]:
for seq in tokenized_texts["input_ids"][:5]:
    print(len(seq))

Самое распространенное решение этой проблемы в NLP — использование дополнения (padding). К счастью, токенизатор из `transformers` может сделать это за нас:

In [ ]:
tokenized_texts = tokenizer(dataset['text'].tolist(), return_tensors="pt", padding=True)

for key, values in tokenized_texts.items():
    values_type = type(values).__name__
    print(f"{key}: {values_type}, {values.shape}")

Однако мы только что добавили много дополнительных элементов в большинство наших последовательностей:

In [ ]:
tokenized_texts["input_ids"]

Обратите внимание, что все последовательности заканчиваются нулями. Мы уже сталкивались с такой проблемой при обучении нашей RNN на предыдущих уроках и решали её, указывая индекс паддинга в `CrossEntropyLoss`, чтобы наша модель не училась предсказывать паддинг. 

Сейчас мы не хотим обучать нашу модель, однако мы работаем с трансформером, который активно использует механизм самовнимания (`self-attention`). Если мы просто добавим эти дополнительные элементы, это может повлиять на наши результаты. И именно здесь в игру вступает `attention_mask`. Она используется именно для того, чтобы исключить паддинг из механизма внимания!

In [ ]:
import matplotlib.pyplot as plt

plt.pcolormesh(tokenized_texts["attention_mask"])
plt.axis("off")
plt.colorbar()
plt.show()

### 1.4. И снова Deep Learning!

Теперь, когда наша модель и входные данные готовы, давайте запустим модель! Однако выполнение на CPU займет несколько минут. Мы можем ускорить этот процесс, используя GPU. Для этого нам нужно разбить наш набор данных на батчи.

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" />

Давайте выделим только ту часть вывода, которая нам нужна. А именно - вывод, соответствующий первому токену каждого предложения. 

`BERT` выполняет классификацию предложений, добавляя токен `[CLS]` (для классификации) в начало каждого предложения. 

Вывод, соответствующий этому токену, можно рассматривать как эмбеддинг всего предложения.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

Мы сохраним эти данные в переменной `features`, так как они будут служить признаками для нашей модели логистической регрессии. Также вспомните, что мы создали переменную `labels` для хранения меток.

In [ ]:
import numpy as np


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

batch_size = 32
features = []
with torch.no_grad():
    for i in range(0, len(texts), batch_size):
        texts_batch = tokenized_texts["input_ids"][i : i + batch_size].to(device)
        masks_batch = tokenized_texts["attention_mask"][i : i + batch_size].to(device)
        # Прогон через BERT
        output = model(texts_batch, masks_batch)
        # Извлечение [CLS]-векторов
        batch_features = output.last_hidden_state[:, 0, :].cpu().numpy()
        features.append(batch_features)

features = np.concatenate(features, axis=0)
features.shape

## 2. Обучение классификации

Давайте теперь разделим наш набор данных на обучающий и тестовый (несмотря на то, что мы используем 2000 предложений из обучающего набора SST2).

In [37]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-train-test-split-sentence-embedding.png" />

Мы могли бы сразу перейти к логистической регрессии с параметрами по умолчанию в `Scikit Learn`, но иногда стоит выполнить поиск наилучшего значения параметра `C`, который определяет силу регуляризации.

In [ ]:
# YOUR CODE HERE
# [EXTRA] Grid search for parameters

Теперь мы обучаем модель логистической регрессии. Если вы выполнили поиск по сетке, можете подставить найденные оптимальные значения гиперпараметров в объявление модели.

In [38]:
import warnings
warnings.simplefilter('ignore')  # Ignore warning on model fitting.

# YOUR CODE HERE

<img src="https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" />

Итак, насколько хорошо наша модель классифицирует предложения? Один из способов - проверить точность на основе тестового набора данных:

In [ ]:
# YOUR CODE HERE

Другой способ оценить модель классификации - построить ROC-кривую и вычислить площадь под ней (AUC).

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

plt.figure(figsize=(10, 6))

proba = lr_clf.predict_proba(train_features)[:, 1]
auc = roc_auc_score(train_labels, proba)
plt.plot(*roc_curve(train_labels, proba)[:2], label=f'train AUC={auc:.4f}')

proba = lr_clf.predict_proba(test_features)[:, 1]
auc = roc_auc_score(test_labels, proba)
plt.plot(*roc_curve(test_labels, proba)[:2], label=f'test AUC={auc:.4f}')

plt.legend()
plt.show()

Насколько хорош этот результат? С чем мы можем его сравнить? Давайте сначала посмотрим на базовый классификатор:

`DummyClassifier` — это очень простая (тупая) модель из `sklearn`, которая не обучается на признаках. Она служит контрольной точкой (baseline), чтобы понять, насколько хорошо работает "реальная" модель по сравнению с чисто случайной стратегией.

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score


clf = DummyClassifier()
scores = cross_val_score(clf, train_features, train_labels)
print(f"Dummy classifier score: {scores.mean():.3f} (+/- {2 * scores.std():.3f})")

Итак, наша модель явно работает лучше базового классификатора. Но насколько она близка к лучшим моделям?

Для справки: [наивысший показатель точности](http://nlpprogress.com/english/sentiment_analysis.html) для этого набора данных в настоящее время составляет **96,8**. DistilBERT можно дообучить для улучшения его результатов в этой задаче — процесс, называемый **тонкой настройкой** (fine-tuning), который обновляет веса BERT для достижения лучшей производительности в этой задаче классификации предложений (которую мы можем назвать downstream task). 
В результате тонкой настройки DistilBERT достигает точности **90,7**. Полноразмерная модель BERT показывает результат **94,9**.

И это всё! Это было хорошее первое знакомство с BERT. Следующим шагом может быть изучение документации и попытка [тонкой настройки](https://huggingface.co/transformers/examples.html#glue) самостоятельно. Вы также можете вернуться и заменить DistilBERT на обычный BERT, чтобы сравнить их работу.

# Взгляд назад

Теперь ваша очередь повторить шаги, описанные выше.

Мы вернемся к первой домашней работе и посмотрим, сможем ли мы немного улучшить результаты. Среднее значение ROC-AUC на тестовой выборке составляло около $0.9$ (с использованием векторных представлений слов).

**Давайте посмотрим, сможем ли мы превзойти этот результат.**

In [ ]:
dataset_url = 'https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/comments_small_dataset/comments.tsv'
dataset = pd.read_csv(dataset_url, sep='\t')
dataset.head()

One last note: this dataset contains some very long sentences, while the vast majority of sequences fall into category of 500 tokens and less:

In [ ]:
texts = dataset["comment_text"].tolist()
tokenized_texts = tokenizer(texts)
ids_lens = list(len(toks) for toks in tokenized_texts["input_ids"])

plt.figure(figsize=(10, 6))
plt.hist(ids_lens)
plt.show()

We already know, how to tackle the problem of different sizes of sequences with padding. However, blind padding here would make pad all the sequenes to the size of the largest one, which seems to be an overkill. In such case it might be sensible to actually truncate the too-long sequences into a fixed length, say 512. And we can do this easily by specifying the `max_length` and `truncation=True` arguments to the tokenizer.

In [ ]:
tokenized_texts = # YOUR CODE HERE

features = # YOUR CODE HERE

In [ ]:
labels = dataset["should_ban"].values
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = # YOUR CODE HERE
lr_clf.fit # YOUR CODE HERE
lr_clf.score # YOUR CODE HERE

In [ ]:
plt.figure(figsize=(10, 6))

proba = lr_clf.predict_proba(train_features)[:, 1]
auc = roc_auc_score(train_labels, proba)
plt.plot(*roc_curve(train_labels, proba)[:2], label=f'train AUC={auc:.4f}')

proba = lr_clf.predict_proba(test_features)[:, 1]
auc = roc_auc_score(test_labels, proba)
plt.plot(*roc_curve(test_labels, proba)[:2], label=f'test AUC={auc:.4f}')

plt.legend()
plt.show()